In [7]:
import numpy as np


In [11]:
t_CLT = [160, 180, 200, 220, 240, 260, 280, 300]
GL_b_ = [ 280, 320, 340, 360, 380, 400, 420, 440, 480, 500, 520, 540, 560, 580, 600 ] 
GL_h_ = [320, 360, 400, 440, 480, 520, 560, 600, 640, 680, 720, 760, 800 ]

In [12]:
plot(t_CLT, GL_b_)

NameError: name 'plot' is not defined